In [ ]:
# Install required dependencies for execution in Google Colab.
# This notebook was developed and tested on a CUDA-enabled runtime.

!pip install -q \
  torch torchvision torchaudio \
  transformers \
  datasets==2.18.0 \
  fsspec==2024.2.0 \
  peft \
  accelerate \
  bitsandbytes \
  evaluate


In [ ]:
# Imports & Environment

import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import datasets
from datasets import load_dataset, concatenate_datasets

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

from peft import LoraConfig, get_peft_model


In [ ]:
# Directory Setup

import os

# Create directories used throughout training.
# - data/processed_safe : placeholder for any preprocessed or cached datasets
# - checkpoints         : stores PEFT (LoRA) adapters and model checkpoints
# - results             : stores training metrics and lightweight artifacts
#
# Using exist_ok=True makes the script idempotent and safe to re-run.
dirs = [
    "data/processed_safe",
    "checkpoints",
    "results"
]

for d in dirs:
    os.makedirs(d, exist_ok=True)


In [ ]:
# Reproducibility Setup

from datasets import load_dataset, Dataset, concatenate_datasets
import random

# Fixed random seed for reproducibility of dataset shuffling and sampling
SEED = 42
random.seed(SEED)


In [ ]:

# Load the EmpatheticDialogues training split.
# This dataset contains multi-turn conversations focused on empathetic responses.
raw_ed = load_dataset("empathetic_dialogues", split="train")

def map_ed(x):
    # Keep only user turns (speaker_idx == 0).
    # Assistant turns are ignored in this simplified formulation.
    if x["speaker_idx"] != 0:
        return {"text": None, "emotion": None, "strategy": None}

    # Format the user utterance as a prompt for the assistant.
    # Emotion and strategy labels are set to -1 since this dataset
    # does not provide explicit supervision for these objectives.
    return {
        "text": f"User: {x['utterance']}\nAssistant:",
        "emotion": -1,
        "strategy": -1
    }

# Apply the mapping function to all examples.
ed = raw_ed.map(map_ed)

# Filter out examples that were dropped during mapping.
ed = ed.filter(lambda x: x["text"] is not None)

# Retain only the columns required for downstream training.
ed = ed.remove_columns(
    [c for c in ed.column_names if c not in ["text", "emotion", "strategy"]]
)

# Sanity check: verify schema and inspect one sample.
print(ed.column_names)
print(ed[0])


In [ ]:
import json

# Mapping from ESConv strategy names to integer class IDs.
# These IDs are used by the auxiliary strategy classification head.
STRATEGY_MAP = {
    "Question": 0,
    "Reflection of feelings": 1,
    "Restatement or Paraphrasing": 2,
    "Providing Suggestions": 3,
    "Affirmation and Reassurance": 4,
    "Information": 5,
    "Others": 6
}

def batched_map_es(batch):
    # Containers for the flattened examples extracted from each conversation
    texts = []
    emotions = []
    strategies = []

    # Each entry in batch["text"] is a JSON-encoded conversation
    for raw_json in batch["text"]:
        data = json.loads(raw_json)

        # Iterate over dialog turns within a conversation
        for turn in data.get("dialog", []):
            # Keep only supporter ("sys") turns that have an associated strategy label
            if turn.get("speaker") == "sys" and "strategy" in turn:
                strat = turn["strategy"]

                # Skip any strategy labels not covered by STRATEGY_MAP
                if strat not in STRATEGY_MAP:
                    continue

                # Use a placeholder user prompt since the original user turn
                # is not explicitly reconstructed in this mapping.
                texts.append("User: <support needed>\nAssistant:")

                # ESConv does not provide explicit emotion labels
                emotions.append(-1)

                # Convert strategy string into its integer class ID
                strategies.append(STRATEGY_MAP[strat])

    # Return a flattened batch suitable for HuggingFace dataset mapping
    return {
        "text": texts,
        "emotion": emotions,
        "strategy": strategies
    }


In [ ]:
# Load the ESConv training split.
# This dataset contains emotional support conversations with strategy annotations.
raw_es = load_dataset("thu-coai/esconv", split="train")

# Apply the batched mapping function to extract supporter turns
# and convert strategy labels into integer IDs.
# All original columns are removed to retain only the processed fields.
es = raw_es.map(
    batched_map_es,
    batched=True,
    remove_columns=raw_es.column_names
)

# Sanity check: verify schema and inspect one processed example.
print(es.column_names)
print(es[0])


In [ ]:
# Load the GoEmotions training split.
# This dataset provides fine-grained emotion labels for single user utterances.
raw_ge = load_dataset("go_emotions", split="train")

def map_ge(x):
    # Select the first emotion label if available.
    # Samples without labels are assigned -1 to indicate missing supervision.
    label = x["labels"][0] if len(x["labels"]) > 0 else -1

    # Format the user utterance as an assistant prompt.
    # Strategy labels are set to -1 since GoEmotions does not provide them.
    return {
        "text": f"User: {x['text']}\nAssistant:",
        "emotion": int(label),
        "strategy": -1
    }

# Apply the mapping function and drop all original columns.
ge = raw_ge.map(map_ge, remove_columns=raw_ge.column_names)

# Sanity check: verify the resulting schema.
print(ge.column_names)


In [ ]:
# Combine the processed EmpatheticDialogues, GoEmotions, and ESConv datasets
# into a single training dataset. Each source contributes a different
# supervision signal (generation prompts, emotion labels, or strategy labels).
dataset = concatenate_datasets([ed, ge, es])

# Sanity checks: inspect dataset size, schema, and one example.
print(dataset)
print(dataset.column_names)
print(dataset[0])

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"  # or 3B if you switched
MAX_LEN = 128

In [ ]:
# Base model used for tokenization and fine-tuning.
# The same tokenizer is reused during training and inference.


# Load the tokenizer associated with the base model.
# The pad token is explicitly set to EOS to avoid padding-related issues
# with causal language models.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token


def tokenize(example):
    # Tokenize the input text prompt.
    # Padding and truncation are applied to ensure fixed-length inputs
    # compatible with batched training.
    t = tokenizer(
        text=example["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

    # Return tokenized inputs along with auxiliary labels.
    # Emotion and strategy labels may be -1 to indicate missing supervision.
    return {
        "input_ids": t["input_ids"],
        "attention_mask": t["attention_mask"],
        "emotion": int(example["emotion"]),
        "strategy": int(example["strategy"])
    }

# Apply tokenization to the full mixed dataset and remove raw text columns.
tokenized = dataset.map(
    tokenize,
    remove_columns=dataset.column_names
)


In [ ]:

# Load the base causal language model with 4-bit quantization
# to reduce memory usage during fine-tuning.
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto"
)

# Configure LoRA for parameter-efficient fine-tuning.
# Only attention projection layers are adapted.
lora = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

# Wrap the base model with LoRA adapters.
model = get_peft_model(base_model, lora)


class AuxHeads(nn.Module):
    def __init__(self, hidden, emo=28, strat=7):
        super().__init__()
        # Linear head for emotion classification
        self.emo = nn.Linear(hidden, emo)
        # Linear head for support strategy classification
        self.strat = nn.Linear(hidden, strat)

    def forward(self, h):
        # Use the hidden state of the final token as a pooled representation
        # for auxiliary classification tasks.
        pooled = h[:, -1, :]
        return self.emo(pooled), self.strat(pooled)

In [ ]:
# Instantiate auxiliary heads and move them to the same device as the model.

aux = AuxHeads(model.config.hidden_size).to(
    device=model.device,
    dtype=model.dtype
)


In [ ]:
# Select a small batch of examples for a forward-pass sanity check.
# This helps verify tensor shapes, device placement, and loss computation
# before launching full training.
batch = tokenized.select(range(2))

# Move inputs and labels to the same device as the model.
input_ids = torch.tensor(batch["input_ids"]).to(model.device)
mask = torch.tensor(batch["attention_mask"]).to(model.device)
emotion = torch.tensor(batch["emotion"]).to(model.device)
strategy = torch.tensor(batch["strategy"]).to(model.device)

# Run a forward pass through the base model and request hidden states
# for use by the auxiliary classification heads.
outputs = model(
    input_ids=input_ids,
    attention_mask=mask,
    output_hidden_states=True
)

# Extract the final hidden layer representations.
hidden = outputs.hidden_states[-1]

# Compute emotion and strategy logits using the auxiliary heads.
emo_logits, strat_logits = aux(hidden)

def masked_ce(logits, targets, ignore_index=-1):
    # Compute cross-entropy loss while ignoring samples
    # with missing supervision (label == ignore_index).
    valid = targets != ignore_index
    if valid.sum() == 0:
        return torch.tensor(0.0, device=logits.device)
    return F.cross_entropy(logits[valid], targets[valid])

# Compute auxiliary losses for emotion and strategy prediction.
emo_loss = masked_ce(emo_logits, emotion)
strat_loss = masked_ce(strat_logits, strategy)

# Print losses as a sanity check to confirm valid forward computation.
print("Emotion loss:", emo_loss.item())
print("Strategy loss:", strat_loss.item())


In [ ]:
# Set the base model and auxiliary heads to training mode.
# This enables training-specific behavior such as dropout.
model.train()
aux.train()


In [ ]:
def masked_ce(logits, targets, ignore_index=-1):
    # Compute cross-entropy loss while ignoring targets marked with ignore_index.
    # This allows mixing examples with and without auxiliary supervision
    # in the same batch.
    valid = targets != ignore_index
    if valid.sum() == 0:
        return torch.tensor(0.0, device=logits.device)
    return F.cross_entropy(logits[valid], targets[valid])

class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Forward pass through the base language model.
        # Hidden states are requested for use by auxiliary heads.
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            output_hidden_states=True
        )

        # Extract the final hidden layer representations.
        hidden = outputs.hidden_states[-1]

        # Compute auxiliary task logits using the shared hidden states.
        emo_logits, strat_logits = aux(hidden)

        # Compute masked auxiliary losses.
        # Samples without emotion or strategy labels are ignored.
        emo_loss = masked_ce(emo_logits, inputs["emotion"])
        strat_loss = masked_ce(strat_logits, inputs["strategy"])

        # Combine auxiliary losses into a single scalar loss.
        # Note: language modeling loss is intentionally excluded here.
        loss = emo_loss + strat_loss

        return (loss, outputs) if return_outputs else loss


In [ ]:
from transformers import default_data_collator

# Use HuggingFace's default data collator to batch inputs.
# This handles padding and tensor conversion for standard fields
# such as input_ids, attention_mask, and auxiliary labels.
def data_collator(features):
    return default_data_collator(features)


In [ ]:
# Define training hyperparameters and runtime configuration.
args = TrainingArguments(
    output_dir="./checkpoints",            # Directory for saving checkpoints and logs
    per_device_train_batch_size=1,         # Small batch size to fit large models in memory
    gradient_accumulation_steps=16,        # Accumulate gradients to simulate a larger batch
    num_train_epochs=1,                    # Single epoch for a time-bounded assignment run
    fp16=False,                            # Disabled to avoid mixed-precision instability
    bf16=False,                            # Disabled for broader hardware compatibility
    logging_steps=50,                      # Log training metrics every N steps
    save_strategy="no",                    # Disable checkpoint saving during training
    report_to="none",                      # Disable external logging integrations
    remove_unused_columns=False            # Ensure auxiliary labels are preserved
)


In [ ]:
# Initialize the custom Trainer with the multi-task loss definition.
# The Trainer orchestrates batching, optimization, and logging.
trainer = MultiTaskTrainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    data_collator=data_collator
)


In [ ]:
# Print data types to verify consistency across model components.
# This is especially important when using quantization or reduced precision
# to avoid silent dtype mismatches during training.
print("Model dtype:", model.dtype)
print("Hidden dtype:", hidden.dtype)
print("Aux dtype:", next(aux.parameters()).dtype)


In [ ]:
# Enable gradient checkpointing to reduce memory usage during training.
# This is important when fine-tuning large models with limited GPU memory.
model.gradient_checkpointing_enable()
model.config.use_cache = False

print("Ready to train")


In [ ]:
train_result = trainer.train()


In [ ]:
# Save the trained LoRA adapter weights.
# These adapters contain all learned parameters from fine-tuning,
# while the base model remains unchanged and is reloaded at inference time.
model.save_pretrained("results/lora_adapter")

# Save the tokenizer to ensure consistency between training and inference.
# This avoids token mismatch issues when loading the model later.
tokenizer.save_pretrained("results/lora_adapter")


# Persist lightweight training metadata for reference.
# This records loss values and logging information produced during training
# and serves as evidence that the training run completed successfully.
with open("results/train_metrics.json", "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)


print("Training complete. LoRA adapters and metrics saved to the results/ directory.")
